In [33]:
import nltk
import random
import numpy as np
import pandas as pd
import pprint, time
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

'''
Reading the input file and storing the 
values of the 3 columns of each row in
a tuple: (<Word>, <POS_TAG>, <CHUNK_TAG>)
'''
f = open("train.txt", "r")
sentence_corpus = []
sentence = []

for line in f:
    line = line.strip()
    if line == "":
        sentence_corpus.append(sentence)
        sentence = []
    else:
        word, pos_tag, _ = line.split(" ")
        #ignoring the chunk tag for this task
        sentence.append((word, pos_tag))
f.close()

# Add the last sentence (if any)
if sentence:
    sentence_corpus.append(sentence)

In [91]:
print(sentence_corpus[:2])

[[('Confidence', 'NN'), ('in', 'IN'), ('the', 'DT'), ('pound', 'NN'), ('is', 'VBZ'), ('widely', 'RB'), ('expected', 'VBN'), ('to', 'TO'), ('take', 'VB'), ('another', 'DT'), ('sharp', 'JJ'), ('dive', 'NN'), ('if', 'IN'), ('trade', 'NN'), ('figures', 'NNS'), ('for', 'IN'), ('September', 'NNP'), (',', ','), ('due', 'JJ'), ('for', 'IN'), ('release', 'NN'), ('tomorrow', 'NN'), (',', ','), ('fail', 'VB'), ('to', 'TO'), ('show', 'VB'), ('a', 'DT'), ('substantial', 'JJ'), ('improvement', 'NN'), ('from', 'IN'), ('July', 'NNP'), ('and', 'CC'), ('August', 'NNP'), ("'s", 'POS'), ('near-record', 'JJ'), ('deficits', 'NNS'), ('.', '.')], [('Chancellor', 'NNP'), ('of', 'IN'), ('the', 'DT'), ('Exchequer', 'NNP'), ('Nigel', 'NNP'), ('Lawson', 'NNP'), ("'s", 'POS'), ('restated', 'VBN'), ('commitment', 'NN'), ('to', 'TO'), ('a', 'DT'), ('firm', 'NN'), ('monetary', 'JJ'), ('policy', 'NN'), ('has', 'VBZ'), ('helped', 'VBN'), ('to', 'TO'), ('prevent', 'VB'), ('a', 'DT'), ('freefall', 'NN'), ('in', 'IN'), ('s

In [ ]:
'''
First implementation: Vertebi Algorithm from scratch
Note: Time consuming: Test data running for more than
      3 Hours.
'''

In [5]:
#Splitting the corpus data into train_data and test_data (validadtion) (80/20 split)
train_set,test_set =train_test_split(sentence_corpus,train_size=0.80,test_size=0.20,random_state = 101)

# List of all the tags in the train and the test set (it may not be unique)
train_tag_corpus = [ t for sentence in train_set for t in sentence ]
test_tag_corpus = [ t for sentence in test_set for t in sentence ]
print(len(train_tag_corpus))
print(len(test_tag_corpus))

170288
41439


In [6]:
print(train_tag_corpus[:20])

[('Besides', 'IN'), ('sacking', 'VBG'), ('other', 'JJ'), ('senior', 'JJ'), ('Politburo', 'NNP'), ('officials', 'NNS'), ('who', 'WP'), ('allied', 'VBD'), ('themselves', 'PRP'), ('with', 'IN'), ('Mr.', 'NNP'), ('Honecker', 'NNP'), (',', ','), ('Mr.', 'NNP'), ('Krenz', 'NNP'), ('could', 'MD'), ('loosen', 'VB'), ('controls', 'NNS'), ('on', 'IN'), ('the', 'DT')]


In [7]:
# Finding number of unique tags and words (Vocabulary)
train_tag_set = {tag for word, tag in train_tag_corpus}
vocab = {word for word, tag in train_tag_corpus}

In [8]:
#Methods to compute transition and emission

'''
prev_tag -> current_tag 
Pr(current_tag | prev_tag) = (# of prev_tag -> current_tag)/(# of prev_tag)
'''
def computeTransition(prev_tag, current_tag):
    tags = [tag for _, tag in train_tag_corpus]
    
    #Count of prev_tag
    cnt_prev_tag = len([tag for tag in tags if tag == prev_tag])
    cnt_prev_curr_tag = 0
    
    for i in range(1, len(tags)):
        if tags[i-1] == prev_tag and tags[i] == current_tag:
            cnt_prev_curr_tag += 1
    
    return cnt_prev_curr_tag / cnt_prev_tag

In [9]:
#The crux of HMM is the emission and transition probabilities

#Transition
transition = np.zeros((len(train_tag_set), len(train_tag_set)), dtype='float32')
train_tag_list = list(train_tag_set)
for i in range(len(train_tag_list)):
    for j in range(len(train_tag_list)):
        transition[i,j] = computeTransition(train_tag_list[i], train_tag_list[j])

In [19]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tag_corpus):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [20]:
tags_df = pd.DataFrame(transition, columns = list(train_tag_list), index=list(train_tag_list))

,DT,:,WDT,UH,),RB,",",VBZ,RP,(,...,IN,PDT,EX,RBS,SYM,POS,'',PRP,VBP,RBR
DT,0.001423,0.000474,0.000203,0.000271,0.000136,0.011925,0.002575,0.008266,0.000000,0.000474,...,0.008469,0.000000,0.000000,0.002846,0.000000,0.000000,0.000000,0.000813,0.000949,0.001355
:,0.152844,0.001185,0.018957,0.000000,0.000000,0.073460,0.000000,0.023697,0.000000,0.000000,...,0.081754,0.001185,0.003555,0.002370,0.000000,0.000000,0.003555,0.035545,0.008294,0.000000
WDT,0.028461,0.001294,0.000000,0.000000,0.000000,0.027167,0.003881,0.285899,0.000000,0.000000,...,0.011643,0.000000,0.001294,0.000000,0.000000,0.001294,0.000000,0.032342,0.155239,0.000000
UH,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.615385,0.000000,0.000000,0.000000,...,0.076923,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000
),0.053719,0.070248,0.000000,0.000000,0.000000,0.020661,0.144628,0.074380,0.000000,0.000000,...,0.132231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012397,0.016529,0.004132
RB,0.051529,0.004341,0.000566,0.000000,0.000189,0.059456,0.092110,0.039826,0.000000,0.000189,...,0.134579,0.000566,0.000755,0.000378,0.000000,0.000566,0.001133,0.010004,0.030389,0.004908
",",0.132940,0.000116,0.034363,0.000000,0.000000,0.046280,0.000000,0.032859,0.000116,0.000116,...,0.099387,0.000000,0.002777,0.001388,0.000000,0.000231,0.056925,0.043272,0.007636,0.000579
VBZ,0.169405,0.005386,0.000269,0.000000,0.000000,0.133854,0.028279,0.001347,0.001616,0.000269,...,0.096687,0.000269,0.000808,0.001616,0.000000,0.000000,0.000000,0.023431,0.001616,0.002155
RP,0.343284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.328358,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(,0.106383,0.000000,0.008511,0.000000,0.000000,0.042553,0.000000,0.000000,0.000000,0.000000,...,0.110638,0.000000,0.004255,0.000000,0.000000,0.000000,0.000000,0.025532,0.000000,0.000000


In [37]:
def viterbi_memoization(words):
    train_bag = train_tag_corpus
    tags = list(set([pair[1] for pair in train_bag]))
    
    # initialize memoization dictionary
    memo = {}
    
    # initialize probability matrix
    T = len(words)
    prob_matrix = np.zeros((T, len(tags)))
    
    # fill in first column of probability matrix
    for i, tag in enumerate(tags):
        if (words[0], tag) in memo:
            emission_p = memo[(words[0], tag)]
        else:
            emission_p = word_given_tag(words[0], tag)[0] / word_given_tag(words[0], tag)[1]
            memo[(words[0], tag)] = emission_p
        prob_matrix[0][i] = tags_df.loc['.', tag] * emission_p
        
    # fill in remaining columns of probability matrix
    for i in range(1, T):
        for j, tag in enumerate(tags):
            max_prob = 0
            for k, prev_tag in enumerate(tags):
                transition_p = tags_df.loc[prev_tag, tag]
                prob = prob_matrix[i-1][k] * transition_p
                if prob > max_prob:
                    max_prob = prob
                    if (words[i], tag) in memo:
                        emission_p = memo[(words[i], tag)]
                    else:
                        emission_p = word_given_tag(words[i], tag)[0] / word_given_tag(words[i], tag)[1]
                        memo[(words[i], tag)] = emission_p
                    prob_matrix[i][j] = max_prob * emission_p
                    
    # backtrack to find optimal sequence of tags
    state = []
    max_prob = max(prob_matrix[-1])
    prev_tag = None
    for i in range(T-1, -1, -1):
        for j, tag in enumerate(tags):
            if prob_matrix[i][j] == max_prob:
                if prev_tag:
                    state.insert(0, prev_tag)
                max_prob /= memo[(words[i], tag)]
                max_prob /= tags_df.loc[prev_tag, tag]
                prev_tag = tag
                break
    
    state.insert(0, prev_tag)
    return list(zip(words, state))

In [39]:
rndom = [random.randint(1,len(test_set)) for x in range(10)]
test_run = [test_set[i] for i in rndom]
test_run_base = [tup for sent in test_run for tup in sent]
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [40]:
tagged_seq = Viterbi_memoization(test_tagged_words)
  
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

#Accuracy of random 10 sentences on the split test data set is 94% using Viterbi.

Time taken in seconds:  227.11126899719238
Viterbi Algorithm Accuracy:  94.3298969072165


In [ ]:
'''
Second Implementation: Using NLTK's hmm
Takes less time and easier to impement
'''

In [43]:
#Creating HMM object
HmmModel = nltk.HiddenMarkovModelTagger.train(train_set)

true_pos_tags = [tag for sentences in test_run for word, tag in sentences]

predicted_pos_tags=[]
for sentences in test_run:
    predicted_pos_tags += [tag for _, tag in HmmModel.tag([word for word, _ in sentences])]

In [44]:
#Accuracy
print (classification_report(true_pos_tags, predicted_pos_tags))
#Accuracy of random 10 sentences on the split test data set is 95% using nltk's hmm

              precision    recall  f1-score   support

           $       1.00      1.00      1.00         2
          ''       1.00      1.00      1.00         2
           ,       0.86      1.00      0.92         6
           .       1.00      1.00      1.00        10
           :       1.00      1.00      1.00         1
          CC       1.00      1.00      1.00         4
          CD       1.00      1.00      1.00        10
          DT       1.00      0.95      0.97        19
          IN       0.95      1.00      0.98        21
          JJ       1.00      0.75      0.86         8
         JJR       0.50      1.00      0.67         1
         JJS       1.00      1.00      1.00         1
          NN       1.00      0.91      0.95        23
         NNP       0.95      0.95      0.95        22
        NNPS       1.00      1.00      1.00         1
         NNS       1.00      0.90      0.95        10
         PRP       1.00      1.00      1.00         5
        PRP$       1.00    

/Users/aman/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aman/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aman/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
true_pos_tags = [tag for sentences in test_set for word, tag in sentences]

predicted_pos_tags=[]
for sentences in test_set:
    predicted_pos_tags += [tag for _, tag in HmmModel.tag([word for word, _ in sentences])]

In [46]:
#Accuracy
print (classification_report(true_pos_tags, predicted_pos_tags))

/Users/aman/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           #       1.00      1.00      1.00         6
           $       0.81      1.00      0.90       311
          ''       0.77      1.00      0.87       297
           (       0.97      0.97      0.97        39
           )       0.64      0.92      0.76        39
           ,       0.98      1.00      0.99      2127
           .       0.95      1.00      0.97      1767
           :       0.99      0.99      0.99       203
          CC       0.97      1.00      0.99      1054
          CD       0.95      0.91      0.93      1599
          DT       0.95      0.99      0.97      3576
          EX       1.00      0.89      0.94        46
          FW       1.00      0.10      0.18        10
          IN       0.97      0.99      0.98      4399
          JJ       0.90      0.86      0.88      2549
         JJR       0.86      0.93      0.89       169
         JJS       0.98      0.87      0.92        75
          MD       0.92    

/Users/aman/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aman/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Accuracy on the split test data set is 93%